<a href="https://colab.research.google.com/github/alesordo/LabAI/blob/main/LabAI2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Imports
# TensorFlow and tf.keras
from tensorflow import keras
from keras.applications import VGG16
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import img_to_array
from keras.utils import load_img
from keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from imutils import paths

# Commonly used modules
import statistics
import pathlib

# Images, plots, display, and visualization
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pickle
import cv2
import csv
from PIL import Image

import os

from tqdm import tqdm

In [2]:
#@title Configuration of directories and variables

# Mounting images directories
BASE_IN = "/content/drive/MyDrive/Archive"
TRAIN_DIR = os.path.sep.join([BASE_IN, "Train"])
TEST_DIR = os.path.sep.join([BASE_IN + "Test"])
PATHS_VAL = os.path.sep.join([BASE_IN + "ValidationPaths.csv"])
ANNOTATION_TRAIN = os.path.sep.join([BASE_IN, "Train.csv"])
ANNOTATION_TEST = os.path.sep.join([BASE_IN, "Test.csv"])

#Mounting output directories
BASE_OUT = os.path.sep.join([BASE_IN, "Output"])
MODEL_PATH = os.path.sep.join([BASE_OUT, "model.h5"])
LB_PATH = os.path.sep.join([BASE_OUT, "lb.pickle"])
PLOTS_PATH = os.path.sep.join([BASE_OUT, "plots"])

# Fixed variables
WIDTH = 50
HEIGHT = 50
N_CLASSES = 43

#Deep learning hyperparameters
BATCH_SIZE = 64
N_CHANNELS = 3
INIT_LR = 1e-4
NUM_EPOCHS = 15


In [ ]:
#@title Loading datasets of images, labels, bounding boxes and images paths

#Initialize list of images, class label, bounding box coordinates, image paths
print("Loading datasets...")
data = []
labels = []
bboxes = []
imagePaths = []

#Load annotation file
with open(ANNOTATION_TRAIN, "r") as csvfile:
    rows = csv.reader(csvfile)
    next(rows)
    #Loop rows
    for row in tqdm(rows):
        #Obtain each data from the csv
        (w, h, startX, startY, endX, endY, label, relativeFilePath) = row

        #Reading complete filepaths and images in OpenCV format
        imagePath = os.path.sep.join([BASE_IN, relativeFilePath])
        image = cv2.imread(imagePath)

        # scale the bounding box coordinates relative to the spatial
        # dimensions of the input image
        startX = float(startX) / float(w)
        startY = float(startY) / float(h)
        endX = float(endX) / float(w)
        endY = float(endY) / float(h)

        # load the image and preprocess it
        image = load_img(imagePath, target_size=(224, 224))
        image = img_to_array(image)

        #debug for future
        # plt.imshow(image.astype(np.uint8))
        # plt.savefig(BASE_IN+"/img.jpg")

        # update our list of data, class labels, bounding boxes, and
        # image paths
        data.append(image)
        labels.append(label)
        bboxes.append((startX, startY, endX, endY))
        imagePaths.append(imagePath)

In [3]:
#@title NEW Loading dataset of labels, bounding boxes and images paths

#Initialize list of images, class label, bounding box coordinates, image paths
print("Loading datasets...")
data = []
labels = []
bboxes = []
imagePaths = []

#Load annotation file
with open(ANNOTATION_TRAIN, "r") as csvfile:
    rows = csv.reader(csvfile)
    next(rows)
    #Loop rows
    for row in tqdm(rows):
        #Obtain each data from the csv
        (w, h, startX, startY, endX, endY, label, relativeFilePath) = row

        #Reading complete filepaths and images in OpenCV format
        imagePath = os.path.sep.join([BASE_IN, relativeFilePath])

        # scale the bounding box coordinates relative to the spatial
        # dimensions of the input image
        startX = float(startX) / float(w)
        startY = float(startY) / float(h)
        endX = float(endX) / float(w)
        endY = float(endY) / float(h)

        #debug for future
        # plt.imshow(image.astype(np.uint8))
        # plt.savefig(BASE_IN+"/img.jpg")

        # update our list of data, class labels, bounding boxes, and
        # image paths
        labels.append(label)
        bboxes.append((startX, startY, endX, endY))
        imagePaths.append(imagePath)

Loading datasets...


39209it [00:00, 211111.34it/s]


In [ ]:
#@title NEW Loading dataset of images

NUM_CATEGORIES = len(os.listdir(TRAIN_DIR))

for img in tqdm(imagePaths):
  try:
      image = cv2.imread(img)
      image_fromarray = Image.fromarray(image, 'RGB')
      resize_image = image_fromarray.resize((HEIGHT, WIDTH))
      data.append(np.array(resize_image))
  except:
      print("Error in " + img)

data = np.array(data, dtype="float32") / 255.0
# Changing the list to numpy array
#data = np.array(data)

  3%|▎         | 1370/39209 [07:40<3:10:33,  3.31it/s]

In [ ]:
#@title Further preprocessing

# convert the data, class labels, bounding boxes, and image paths to NumPy arrays, scaling the input pixel from the range [0, 255] to [0, 1]
#data = np.array(data, dtype="float32") / 255.0
labels = np.array(labels)
bboxes = np.array(bboxes, dtype="float32")
imagePaths = np.array(imagePaths)

# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [ ]:
#@title Partition data into training and validation - EXECUTE ONCE, NOT BECAUSE IT CHANGES FILES, BECAUSE IT SAVES VALIDATION PATHS

# partition the data into training and testing splits using 80% of the data for training and the remaining 20% for testing
split = train_test_split(data, labels, bboxes, imagePaths,
	test_size=0.20, random_state=42)

# unpack the data split
(trainImages, valImages) = split[:2]
(trainLabels, valLabels) = split[2:4]
(trainBBoxes, valBBoxes) = split[4:6]
(trainPaths, valPaths) = split[6:]

# write the testing image paths to disk so that we can use then when evaluating/testing our object detector
print("[INFO] saving testing image paths...")
f = open(PATHS_VAL, "w")
f.write("\n".join(valPaths))
f.close()

In [ ]:
#@title Initialize VGG16

# load the VGG16 network, ensuring the head FC layers are left off
vgg = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(HEIGHT, WIDTH, 3)))

# freeze all VGG layers so they will *not* be updated during the training process
vgg.trainable = False

# flatten the max-pooling output of VGG
flatten = vgg.output
flatten = Flatten()(flatten)

In [ ]:
#@title Constructing layer head with two branches

# construct a fully-connected layer header to output the predicted bounding box coordinates
bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(4, activation="sigmoid",
	name="bounding_box")(bboxHead)
 
# construct a second fully-connected layer head, this one to predict the class label
softmaxHead = Dense(512, activation="relu")(flatten)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(512, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(len(lb.classes_), activation="softmax",
	name="class_label")(softmaxHead)
 
# put together our model which accept an input image and then output bounding box coordinates and a class label
model = Model(
	inputs=vgg.input,
	outputs=(bboxHead, softmaxHead))

In [ ]:
#@title Defining dictionaries for losses and compilation of the model

# define a dictionary to set the loss methods -- categorical cross-entropy for the class label head and mean absolute error for the bounding box head
losses = {
	"class_label": "categorical_crossentropy",
	"bounding_box": "mean_squared_error",
}

# define a dictionary that specifies the weights per loss (both the class label and bounding box outputs will receive equal weight)
lossWeights = {
	"class_label": 1.0,
	"bounding_box": 1.0
}

# initialize the optimizer, compile the model, and show the model summary
opt = Adam(lr=INIT_LR)
model.compile(loss=losses, optimizer=opt, metrics=["accuracy"], loss_weights=lossWeights)
print(model.summary())

In [ ]:
#@title Defining dictionaries that represent training and validation set

# construct a dictionary for our target training outputs
trainTargets = {
	"class_label": trainLabels,
	"bounding_box": trainBBoxes
}
# construct a second dictionary, this one for our target testing
# outputs
valTargets = {
	"class_label": valLabels,
	"bounding_box": valBBoxes
}

In [ ]:
#@title Training network

# train the network for bounding box regression and class label prediction
print("[INFO] training model...")
H = model.fit(
	trainImages, trainTargets,
	validation_data=(valImages, valTargets),
	batch_size=BATCH_SIZE,
	epochs=NUM_EPOCHS,
	verbose=1)

# serialize the model to disk
print("[INFO] saving object detector model...")
model.save(MODEL_PATH, save_format="h5")

# serialize the label binarizer to disk
print("[INFO] saving label binarizer...")
f = open(LB_PATH, "wb")
f.write(pickle.dumps(lb))
f.close()

In [ ]:
#@title Plotting losses and accuracy

# plot the total loss, label loss, and bounding box loss
lossNames = ["loss", "class_label_loss", "bounding_box_loss"]
N = np.arange(0, NUM_EPOCHS)
plt.style.use("ggplot")
(fig, ax) = plt.subplots(3, 1, figsize=(13, 13))

# loop over the loss names
for (i, l) in enumerate(lossNames):
	# plot the loss for both the training and validation data
	title = "Loss for {}".format(l) if l != "loss" else "Total loss"
	ax[i].set_title(title)
	ax[i].set_xlabel("Epoch #")
	ax[i].set_ylabel("Loss")
	ax[i].plot(N, H.history[l], label=l)
	ax[i].plot(N, H.history["val_" + l], label="val_" + l)
	ax[i].legend()
 
# save the losses figure and create a new figure for the accuracies
plt.tight_layout()
plotPath = os.path.sep.join([PLOTS_PATH, "losses.png"])
plt.savefig(plotPath)
plt.close()

# create a new figure for the accuracies
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["class_label_accuracy"],
	label="class_label_train_acc")
plt.plot(N, H.history["val_class_label_accuracy"],
	label="val_class_label_acc")
plt.title("Class Label Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend(loc="lower left")

# save the accuracies plot
plotPath = os.path.sep.join([PLOTS_PATH, "accs.png"])
plt.savefig(plotPath)